In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torch.nn as nn
import torch.optim as optim

# 데이터 로딩
def load_data(train_dir, test_dir):
    transform = transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    train_dataset = ImageFolder(root=train_dir, transform=transform)
    test_dataset = ImageFolder(root=test_dir, transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

    return train_loader, test_loader


In [2]:

#CNN 모델
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5) #입력채널 1개, 출력채널 10개
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)#입력채널 10개, 출력채널 20개
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = nn.functional.relu(nn.functional.max_pool2d(self.conv1(x), 2))
        x = nn.functional.relu(nn.functional.max_pool2d(self.conv2(x), 2))
        x = x.view(-1, 320)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return nn.functional.log_softmax(x, dim=1)


In [4]:

# 훈련 및 평가
def train_and_evaluate(model, train_loader, test_loader, epochs=10):
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
    criterion = nn.CrossEntropyLoss() #loss func

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for i, (data, target) in enumerate(train_loader, 0):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 100 == 99:  # 매 100 미니배치마다 출력
                print(f'Epoch {epoch + 1}, Batch {i + 1}, Loss: {running_loss / 100:.4f}')
                running_loss = 0.0

        # 에포크의 끝에서 테스트 데이터에 대한 정확도 출력
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)
                outputs = model(data)
                _, predicted = torch.max(outputs.data, 1)
                total += target.size(0)
                correct += (predicted == target).sum().item()
        print(f'End of Epoch {epoch + 1}, Accuracy: {100 * correct / total}%')


In [6]:
train_dir = 'C:/Users/koo/my_workspace/privacy_term_project/mnist_btm/training'
test_dir = 'C:/Users/koo/my_workspace/privacy_term_project/mnist_btm/testing'

train_loader, test_loader = load_data(train_dir, test_dir)

#model init
model = SimpleCNN()
 
train_and_evaluate(model, train_loader, test_loader, epochs=10)

# model save
torch.save(model, 'C:/Users/koo/my_workspace/privacy_term_project/mnist_cnn.pth')

Epoch 1, Batch 100, Loss: 1.6331
Epoch 1, Batch 200, Loss: 0.3191
Epoch 1, Batch 300, Loss: 0.1790
Epoch 1, Batch 400, Loss: 0.1446
Epoch 1, Batch 500, Loss: 0.1192
Epoch 1, Batch 600, Loss: 0.1125
Epoch 1, Batch 700, Loss: 0.0861
Epoch 1, Batch 800, Loss: 0.0886
Epoch 1, Batch 900, Loss: 0.0795
End of Epoch 1, Accuracy: 97.8%
Epoch 2, Batch 100, Loss: 0.0686
Epoch 2, Batch 200, Loss: 0.0670
Epoch 2, Batch 300, Loss: 0.0626
Epoch 2, Batch 400, Loss: 0.0647
Epoch 2, Batch 500, Loss: 0.0627
Epoch 2, Batch 600, Loss: 0.0588
Epoch 2, Batch 700, Loss: 0.0660
Epoch 2, Batch 800, Loss: 0.0525
Epoch 2, Batch 900, Loss: 0.0611
End of Epoch 2, Accuracy: 98.5%
Epoch 3, Batch 100, Loss: 0.0434
Epoch 3, Batch 200, Loss: 0.0411
Epoch 3, Batch 300, Loss: 0.0499
Epoch 3, Batch 400, Loss: 0.0425
Epoch 3, Batch 500, Loss: 0.0432
Epoch 3, Batch 600, Loss: 0.0437
Epoch 3, Batch 700, Loss: 0.0504
Epoch 3, Batch 800, Loss: 0.0544
Epoch 3, Batch 900, Loss: 0.0407
End of Epoch 3, Accuracy: 98.61%
Epoch 4, Bat

In [11]:
torch.save(model.state_dict(), 'C:/Users/koo/my_workspace/privacy_term_project/mnist_cnn.pth')